In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")

In [3]:
from ds_exploration.data_download import download_tiles

Register at https://scihub.copernicus.eu/dhus/#/home

Then enter your credentials in a file .netrc in your user home directory in the following form:

```
machine scihub.copernicus.eu
login <your username>
password <your password>
```

## Tiles to download

In [4]:
labels = [
    {'date': ('20200928', '20200930'), 'tile_name': '10SEH'},
    {'date': ('20201001', '20201009'), 'tile_name': '10TDK'},
    {'date': ('20200928', '20200930'), 'tile_name': '10TDL'},
    {'date': ('20200627', '20200629'), 'tile_name': '10SFG'},
    {'date': ('20200714', '20200716'), 'tile_name': '10SGF'},
    {'date': ('20200719', '20200721'), 'tile_name': '10SGF'},
    {'date': ('20200722', '20200724'), 'tile_name': '10TFK'},
    {'date': ('20200816', '20200818'), 'tile_name': '10TGK'},
    {'date': ('20200722', '20200724'), 'tile_name': '10TFL'},
    {'date': ('20200722', '20200724'), 'tile_name': '10TFM'},
    {'date': ('20200909', '20200911'), 'tile_name': '11SMT'},
    {'date': ('20200914', '20200916'), 'tile_name': '11SMT'},
    {'date': ('20200909', '20200911'), 'tile_name': '11SMU'},
    {'date': ('20200914', '20200916'), 'tile_name': '11SMU'},
    {'date': ('20200902', '20200904'), 'tile_name': '11SLA'},
    {'date': ('20200904', '20200906'), 'tile_name': '11SLA'},
    {'date': ('20200907', '20200909'), 'tile_name': '11SLA'},
    {'date': ('20200909', '20200911'), 'tile_name': '11SLA'},
    {'date': ('20200912', '20200914'), 'tile_name': '11SLA'},
    {'date': ('20200914', '20200916'), 'tile_name': '11SLA'},
    {'date': ('20200917', '20200919'), 'tile_name': '11SLA'},
    {'date': ('20200919', '20200921'), 'tile_name': '11SLA'},
    {'date': ('20200922', '20200924'), 'tile_name': '11SLA'},
    {'date': ('20200924', '20200926'), 'tile_name': '11SLA'},
    {'date': ('20200927', '20200929'), 'tile_name': '11SLA'},
    {'date': ('20200929', '20201001'), 'tile_name': '11SLA'},
    {'date': ('20201002', '20201004'), 'tile_name': '11SLA'},
    {'date': ('20201007', '20201009'), 'tile_name': '11SLA'},
    {'date': ('20201009', '20201011'), 'tile_name': '11SLA'},
    {'date': ('20201012', '20201014'), 'tile_name': '11SLA'},
    {'date': ('20201014', '20201016'), 'tile_name': '11SLA'},
    {'date': ('20201017', '20201019'), 'tile_name': '11SLA'},
    {'date': ('20201019', '20201021'), 'tile_name': '11SLA'},
    {'date': ('20201022', '20201024'), 'tile_name': '11SLA'},
    {'date': ('20201024', '20201026'), 'tile_name': '11SLA'},
    {'date': ('20201027', '20201029'), 'tile_name': '11SLA'},
    {'date': ('20201029', '20201031'), 'tile_name': '11SLA'},
    {'date': ('20201101', '20201103'), 'tile_name': '11SLA'},
    {'date': ('20201103', '20201105'), 'tile_name': '11SLA'},
    {'date': ('20200907', '20200909'), 'tile_name': '11SKB'},
    {'date': ('20200912', '20200914'), 'tile_name': '11SKB'},
    {'date': ('20200922', '20200924'), 'tile_name': '11SKB'},
    {'date': ('20200927', '20200929'), 'tile_name': '11SKB'},
    {'date': ('20201002', '20201004'), 'tile_name': '11SKB'},
    {'date': ('20201012', '20201014'), 'tile_name': '11SKB'},
    {'date': ('20200907', '20200909'), 'tile_name': '11SLB'},
    {'date': ('20200912', '20200914'), 'tile_name': '11SLB'},
    {'date': ('20200922', '20200924'), 'tile_name': '11SLB'},
    {'date': ('20200927', '20200929'), 'tile_name': '11SLB'},
    {'date': ('20201002', '20201004'), 'tile_name': '11SLB'},
    {'date': ('20201012', '20201014'), 'tile_name': '11SLB'},
    {'date': ('20201017', '20201019'), 'tile_name': '11SLB'},
    {'date': ('20201022', '20201024'), 'tile_name': '11SLB'},
    {'date': ('20201027', '20201029'), 'tile_name': '11SLB'},
    {'date': ('20201101', '20201103'), 'tile_name': '11SLB'},
    {'date': ('20200907', '20200909'), 'tile_name': '11SKB'},
    {'date': ('20200912', '20200914'), 'tile_name': '11SKB'},
    {'date': ('20200903', '20200905'), 'tile_name': '10TDK'},
    {'date': ('20200908', '20200910'), 'tile_name': '10TDK'},
    {'date': ('20200913', '20200915'), 'tile_name': '10TDK'},
    {'date': ('20200918', '20200920'), 'tile_name': '10TDK'},
    {'date': ('20200928', '20200930'), 'tile_name': '10TDK'},
    {'date': ('20200903', '20200905'), 'tile_name': '10TDL'},
    {'date': ('20200908', '20200910'), 'tile_name': '10TDL'},
    {'date': ('20200913', '20200915'), 'tile_name': '10TDL'},
    {'date': ('20200903', '20200904'), 'tile_name': '10TEK'},
    {'date': ('20200905', '20200907'), 'tile_name': '10TEK'},
    {'date': ('20200913', '20200915'), 'tile_name': '10TEK'},
    {'date': ('20200924', '20200926'), 'tile_name': '10TEK'},
    {'date': ('20200928', '20200930'), 'tile_name': '10TEK'},
    {'date': ('20200910', '20200912'), 'tile_name': '10TFK'},
    {'date': ('20200915', '20200917'), 'tile_name': '10TFK'},
    {'date': ('20200903', '20200905'), 'tile_name': '10TEQ'},
    {'date': ('20200908', '20200910'), 'tile_name': '10TEQ'},
    {'date': ('20200913', '20200915'), 'tile_name': '10TEQ'},
    {'date': ('20200903', '20200905'), 'tile_name': '10TFS'},
    {'date': ('20200801', '20200802'), 'tile_name': '11SNT'},
    {'date': ('20200803', '20200804'), 'tile_name': '11SNT'},
    {'date': ('20200806', '20200807'), 'tile_name': '11SNT'},
]

In [5]:
d = download_tiles(labels[0:1], "data", None, None, nodefilter=None)

Download tile name: 10SEH for 20200928, 20200930, with filter None


Downloading: 100%|██████████| 357M/357M [01:04<00:00, 5.54MB/s] 
MD5 checksumming: 100%|██████████| 357M/357M [00:00<00:00, 541MB/s] 


Downloaded 1 tiles, extracting and combining data
Saving 100 jpg images into data/S2B_MSIL1C_20200929T190109_N0209_R013_T10SEH_20200929T213343 folder


In [6]:
d.memory_usage(deep=True)

Index                   93
title                  117
date                     8
footprint              447
url                    155
quicklook_url          177
path                   126
rgb              723362544
b12               60280328
dtype: int64